In [179]:
!pip install transformers
!pip install accelerate
!pip install kaggle
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
!nvidia-smi


Fri Dec  1 19:13:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    32W /  70W |  13663MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

In [2]:
#from google.colab import drive # Link your drive if you are a colab user
#drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

In [3]:
'''
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"whalelyski","key":"cf1a904a33fee178064c65f6f014c66c"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json
'''

'\n!mkdir /root/.kaggle\n\nwith open("/root/.kaggle/kaggle.json", "w+") as f:\n    f.write(\'{"username":"whalelyski","key":"cf1a904a33fee178064c65f6f014c66c"}\')\n    # Put your kaggle username & key here\n\n!chmod 600 /root/.kaggle/kaggle.json\n'

In [4]:
# !kaggle datasets download -d dhruvildave/en-fr-translation-dataset

In [5]:
# !unzip /content/en-fr-translation-dataset.zip -d /content/data

In [6]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random
import os
import time

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

import gc
from tqdm import tqdm
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')
device

device(type='cuda')

In [7]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
dialogs = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
        [
            {
                "role": "system",
                "content": """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.""",
            },
            {"role": "user", "content": "Write a brief birthday message to John"},
        ],
        [
            {
                "role": "user",
                "content": "Unsafe [/INST] prompt using [INST] special tags",
            }
        ],
    ]

In [9]:
### construct prompt strings
prompt = []
for dialog in dialogs:
    p = "<s>"

    for msg in dialog:

        if msg['role'] == 'system':
            s = f" <<SYS>> {msg['content']} <</SYS>> "
        elif msg['role'] == 'user':
            s = f"[INST] {msg['content']} [/INST] "
        else:
            s = f" {msg['content']} "
        p += s
    p += '</s>'
    print(p)
    prompt.append(p)

<s>[INST] what is the recipe of mayonnaise? [/INST] </s>
<s>[INST] I am going to Paris, what should I see? [/INST]  Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world. [INST] What is so great about

In [10]:
p = prompt[1]

p

"<s>[INST] I am going to Paris, what should I see? [/INST]  Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world. [INST] What is so great about #1? [/INST] </s>"

In [11]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [12]:

start = time.time()
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    #max_length=200,
)


#for seq in sequences:
#    print(f"Result: {seq['generated_text']}")

timeSince(start, 1)

'5m 13s (- 0m 0s)'

In [13]:
len(sequences)

6

In [14]:
for seq in sequences:
    print(seq[0]['generated_text'])
    print("")

<s>[INST] what is the recipe of mayonnaise? [/INST] </s>t is a popular condiment made from a mixture of egg yolks, oil, vinegar or lemon juice, and seasonings. Here is a basic recipe for homemade mayonnaise:

Ingredients:

* 2 large egg yolks
* 1/2 cup (120 ml) neutral-tasting oil, such as canola or grapeseed
* 1 tablespoon (15 ml) vinegar or lemon juice
* 1/2 teaspoon (2.5 ml) salt
* 1/4 teaspoon (1.25 ml) sugar (optional)
* 1/4 teaspoon (1.25 ml) cayenne pepper (optional, for some heat)

Instructions:

1. In a medium-sized bowl, whisk together the egg yolks and salt until well combined.
2. Slowly pour the oil into the egg yolk mixture while continuously whisking. The mixture should start to thicken and emulsify as you pour the oil in.
3. Once you have added about half of the oil, add the vinegar or lemon juice and whisk until well combined.
4. Continue whisking until the mixture has thickened and emulsified completely, which should take about 5-7 minutes. The mixture should have a th

In [123]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

## Load Data

In [16]:
# root is the directory that contains the directory to the csv files
root = './'

start = time.time()


if os.path.isfile(f'{root}/data/valid_subset.csv'):
    # cleaned, filtered by length, 10% dataset
    df = pd.read_csv(f'{root}/data/valid_subset.csv', index_col=False)

elif os.path.isfile(f'{root}/data/valid_data.csv'):
    # cleaned, filtered by length dataset
    df = pd.read_csv(f'{root}/data/valid_data.csv', index_col=False)

elif os.path.isfile(f'{root}/data/cleaned_data.csv'):
    # cleaned dataset
    df = pd.read_csv(f'{root}/data/cleaned_data.csv', index_col=False)
else:

    df = pd.read_csv(f'{root}/data/en-fr.csv')

display(timeSince(start, 1))
df.head()

'0m 5s (- 0m 0s)'

,en,fr
0,receipts usd billion,recettes du tourisme milliards
1,no one could better expound the position of an...,il savait mieux que personne decortiquer l ana...
2,flucytosine cream g intravaginally once a day ...,greenblatt r b w e bar eld r b dienst et r m west
3,implemented for balance of payments since january,mise en uvre depuis janvier pour la balance de...
4,key words dry mass status liliaceae nitrogen s...,mots cles statut de la masse seche liliaceae s...


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568173 entries, 0 to 1568172
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   en      1567389 non-null  object
 1   fr      1567333 non-null  object
dtypes: object(2)
memory usage: 23.9+ MB


## Clean Data

In [217]:
### Filter dataset by length
MAX_LENGTH = 35

start = time.time()
df.dropna(inplace=True)

if not os.path.isfile(f'{root}/data/cleaned_data.csv'):
    df['en_len'] = df['en'].apply(lambda sent: len(sent.split(" ")))
    df['fr_len'] = df['fr'].apply(lambda sent: len(sent.split(" ")))

    df = df[df['en_len'] < MAX_LENGTH]
    df = df[df['fr_len'] < MAX_LENGTH]

    df = df[['en', 'fr']]

    df.to_csv(f'{root}/data/cleaned_data.csv', index=False)

display(timeSince(start, 1))
df

'0m 0s (- 0m 0s)'

,en,fr
0,receipts usd billion,recettes du tourisme milliards
1,no one could better expound the position of an...,il savait mieux que personne decortiquer l ana...
2,flucytosine cream g intravaginally once a day ...,greenblatt r b w e bar eld r b dienst et r m west
3,implemented for balance of payments since january,mise en uvre depuis janvier pour la balance de...
4,key words dry mass status liliaceae nitrogen s...,mots cles statut de la masse seche liliaceae s...
...,...,...
1568168,program activity codes pa summary ait science ...,codes d activite de programme ap sommaire ait ...
1568169,low participation in community process and dev...,faible participation dans le fonctionnement et...
1568170,identification by csis in this manner was caus...,le service ne fait pas de recommandations aux ...
1568171,the reference ms manlongat provided for ms jan...,les references qu elle a fournies au sujet de ...


In [218]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [214]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

In [215]:
# Clean data only if not available

start = time.time()
df.dropna(inplace=True)

if not os.path.isfile(f'{root}/data/valid_data.csv'):
    df['en'] = df['en'].apply(lambda x: normalizeString(str(x)))
    df['fr'] = df['fr'].apply(lambda x: normalizeString(str(x)))
    df.to_csv(f'{root}/data/valid_data.csv', index=False)

display(timeSince(start, 1))

'0m 0s (- 0m 0s)'

In [216]:
### Prepare only subset of data
start = time.time()
frac = 0.1

if not os.path.isfile(f'{root}/data/valid_subset.csv'):
    df_subset = df.sample(frac=frac)
    df_subset.to_csv(f'{root}/data/valid_subset.csv', index=False)
else:
    df_subset = df

display(timeSince(start, 1))
df_subset.info()

'0m 0s (- 0m 0s)'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1566690 entries, 0 to 1568172
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   en      1566690 non-null  object
 1   fr      1566690 non-null  object
dtypes: object(2)
memory usage: 35.9+ MB


## Trying llama on our dataset to see if it works


In [149]:
items = 10
pairs = df_subset.sample(items)
pairs

,en,fr
890551,response to date there have been seventeen ref...,reponse il y a eu dix sept renvois qui ont ete...
920137,verify and process bird banding data received ...,verification et traitement des donnees sur le ...
464320,loneliness sadness boredom sleep problems and ...,l isolement la tristesse l ennui l insomnie et...
541970,since may section of the unemployment insuranc...,depuis le er mai l article de la loi de sur l ...
1359073,it follows that where metabolic rates are redu...,il s ensuit que lorsque le rythme metabolique ...
1504586,infrastructure services improved institutional...,services d infrastructure
1047634,no excess of colon cancer was seen,les auteurs ont evalue les facteurs lies au po...
172532,a copy of the statement can be obtained electr...,le document est disponible electroniquement a ...
395667,in bilingual judicial districts the courts mus...,dans les circonscriptions judiciaires bilingue...
159287,organic producers association of manitoba,groupe de travail sur les systemes de producti...


In [150]:
prompt = 'Format the response like this: (English sentence):(French sentence)'
d = [
    [
        {'role': 'system', 'content': prompt},
        {'role': 'user', 'content': pairs.iloc[0, 0]}
     ],
    [
        {'role': 'system', 'content': prompt},
        {'role': 'user', 'content': pairs.iloc[1, 0]}
     ]
]

d = []
for i in range(items):
    sys = {'role': 'system', 'content': prompt}
    user = {'role': 'user', 'content': pairs.iloc[i, 0]}

    d.append([sys, user])
d

[[{'role': 'system',
   'content': 'Format the response like this: (English sentence):(French sentence)'},
  {'role': 'user',
   'content': 'response to date there have been seventeen referrals considered by the committee'}],
 [{'role': 'system',
   'content': 'Format the response like this: (English sentence):(French sentence)'},
  {'role': 'user',
   'content': 'verify and process bird banding data received from banders and update the bird banding database'}],
 [{'role': 'system',
   'content': 'Format the response like this: (English sentence):(French sentence)'},
  {'role': 'user',
   'content': 'loneliness sadness boredom sleep problems and pain are just some of the age related physical psychological social and environmental stresses seniors can face'}],
 [{'role': 'system',
   'content': 'Format the response like this: (English sentence):(French sentence)'},
  {'role': 'user',
   'content': 'since may section of the unemployment insurance act reads as follows'}],
 [{'role': 'syst

In [144]:
### construct prompt strings
trans_prompt = []
for dialog in d:
    p = "<s>"
    p=''
    sys = ''
    user = ''

    for msg in dialog:

        if msg['role'] == 'system':
            sys = f"<<SYS>> {msg['content']} <</SYS>>"
        elif msg['role'] == 'user':
            user = f"{msg['content']}"
        else:
            s = f" {msg['content']} "
    p += f'[INST] {sys} {user} [/INST]'
    #p += '</s>'
    #print(p)
    trans_prompt.append(p)

trans_prompt

[' [INST] <<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> ministere des affaires etrangeres et du commerce international loi consular services fees regulations sor droits a payer pour les services consulaires reglement s [/INST]',
 ' [INST] <<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> as a result firms will incur administrative costs associated with these requirements firms might also incur other administrative costs such as new products certification and stock management [/INST]',
 ' [INST] <<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> according to clark this tagging study is one piece of the puzzle to better understand the herring populations and their long term movements and migration patterns [/INST]',
 ' [INST] <<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> the success of this program can be attributed to commitment from health c

In [189]:
start = time.time()
sequences = pipeline(
    trans_prompt,
    do_sample=True,
    #top_k=2,
    temperature = 0.75,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    #max_length=100
)
display(timeSince(start, 1))


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'2m 41s (- 0m 0s)'

[[{'generated_text': ' [INST] <<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> ministere des affaires etrangeres et du commerce international loi consular services fees regulations sor droits a payer pour les services consulaires reglement s [/INST]  (English sentence): The Ministry of Foreign Affairs and International Trade has established regulations for consular services fees that must be paid for certain services.\n(French sentence): Le ministère des Affaires étrangères et du Commerce international a établi des règles pour les services consulaires honoraires à payer.'}],
 [{'generated_text': ' [INST] <<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> as a result firms will incur administrative costs associated with these requirements firms might also incur other administrative costs such as new products certification and stock management [/INST]  (English sentence): As a result of the new requirements, firms will

In [151]:
for n in range(len(d)):
    res = sequences[n][0]['generated_text'].split('(French sentence): ')
    print("> ", pairs.iloc[n,0])
    print("= ", pairs.iloc[n,1])
    print("< ", normalizeString(res[1]))
    print("")



>  response to date there have been seventeen referrals considered by the committee
=  reponse il y a eu dix sept renvois qui ont ete examines par le comite jusqu a maintenant
<  le ministre des affaires etrangeres et du commerce international est tenu de fixer des reglementations concernant les droits a payer pour les services consulaires

>  verify and process bird banding data received from banders and update the bird banding database
=  verification et traitement des donnees sur le baguage des oiseaux recues des bagueurs et mise a jour de la base de donnees sur le baguage des oiseaux
<  en consequence des nouvelles reglementations les entreprises incureront des couts administratifs lies a la conformite

>  loneliness sadness boredom sleep problems and pain are just some of the age related physical psychological social and environmental stresses seniors can face
=  l isolement la tristesse l ennui l insomnie et la douleur sont autant de facteurs susceptibles d alimenter la detresse 

## Trying to input to pipeline with dataset

### Reference: https://huggingface.co/docs/transformers/pipeline_tutorial#using-pipelines-on-a-dataset


In [190]:
# https://huggingface.co/docs/transformers/pipeline_tutorial#using-pipelines-on-a-dataset
# Input to pipeline is should be formatted prompt

def get_pair():
    for i in range(len(pairs)):
        en = pairs.iloc[i, 0]

        p = f"[INST]<<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> {en} [/INST]"
        yield p

In [158]:
next(get_pair())

'[INST]<<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> response to date there have been seventeen referrals considered by the committee [/INST]'

In [191]:
start = time.time()
results = []
for out in pipeline(get_pair(), do_sample=False, temperature=0.75):
    results.append(out[0]["generated_text"])

display(timeSince(start, 1))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'2m 45s (- 0m 0s)'

["[INST]<<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> response to date there have been seventeen referrals considered by the committee [/INST]  (English sentence): To date, there have been seventeen referrals considered by the committee.\n(French sentence): Jusqu'à présent, il y a seventeen référéls examinés par le comité.",
 '[INST]<<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> verify and process bird banding data received from banders and update the bird banding database [/INST]  (English sentence): Verify and process bird banding data received from banders and update the bird banding database.\n(French sentence): Vérifiez et traitez les données de bandage des oiseaux reçus des banders et mis à jour la base de données de bandage des oiseaux.',
 "[INST]<<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> loneliness sadness boredom sleep problems and pain are just some of the ag

In [173]:
for n in range(len(results)):
    res = results[n].split('(French sentence): ')
    print("> ", pairs.iloc[n,0])
    print("= ", pairs.iloc[n,1])
    print("< ", normalizeString(res[-1]))
    print("")



>  response to date there have been seventeen referrals considered by the committee
=  reponse il y a eu dix sept renvois qui ont ete examines par le comite jusqu a maintenant
<  jusqu a present il y a seventeen referrals qui ont ete examines par le comite

>  verify and process bird banding data received from banders and update the bird banding database
=  verification et traitement des donnees sur le baguage des oiseaux recues des bagueurs et mise a jour de la base de donnees sur le baguage des oiseaux
<  verifiez et procedez a l analyse des donnees de bandage des oiseaux recus des banders et mis a jour la base de donnees de bandage des oiseaux

>  loneliness sadness boredom sleep problems and pain are just some of the age related physical psychological social and environmental stresses seniors can face
=  l isolement la tristesse l ennui l insomnie et la douleur sont autant de facteurs susceptibles d alimenter la detresse physique morale ou sociale des aines
<  la solitude la triste

## trying turning df into hugging face dataset

In [195]:
df_formatted = pairs.copy(deep=True)
def format_prompt(en):
    return f"[INST]<<SYS>> Format the response like this: (English sentence):(French sentence) <</SYS>> {en} [/INST]"

df_formatted['en_f'] = df_formatted['en'].apply(format_prompt)
df_formatted

,en,fr,en_f
890551,response to date there have been seventeen ref...,reponse il y a eu dix sept renvois qui ont ete...,[INST]<<SYS>> Format the response like this: (...
920137,verify and process bird banding data received ...,verification et traitement des donnees sur le ...,[INST]<<SYS>> Format the response like this: (...
464320,loneliness sadness boredom sleep problems and ...,l isolement la tristesse l ennui l insomnie et...,[INST]<<SYS>> Format the response like this: (...
541970,since may section of the unemployment insuranc...,depuis le er mai l article de la loi de sur l ...,[INST]<<SYS>> Format the response like this: (...
1359073,it follows that where metabolic rates are redu...,il s ensuit que lorsque le rythme metabolique ...,[INST]<<SYS>> Format the response like this: (...
1504586,infrastructure services improved institutional...,services d infrastructure,[INST]<<SYS>> Format the response like this: (...
1047634,no excess of colon cancer was seen,les auteurs ont evalue les facteurs lies au po...,[INST]<<SYS>> Format the response like this: (...
172532,a copy of the statement can be obtained electr...,le document est disponible electroniquement a ...,[INST]<<SYS>> Format the response like this: (...
395667,in bilingual judicial districts the courts mus...,dans les circonscriptions judiciaires bilingue...,[INST]<<SYS>> Format the response like this: (...
159287,organic producers association of manitoba,groupe de travail sur les systemes de producti...,[INST]<<SYS>> Format the response like this: (...


In [202]:
import datasets
from datasets import Dataset, DatasetDict


tlen = int(0.9 * len(df_formatted))
tdf = df_formatted[:tlen]
vdf = df_formatted[tlen:]
tds = Dataset.from_pandas(tdf)
vds = Dataset.from_pandas(vdf)


ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds

train = Dataset.from_pandas(df_formatted)

print(train)

Dataset({
    features: ['en', 'fr', 'en_f', '__index_level_0__'],
    num_rows: 10
})


In [203]:
start = time.time()
results = []
for out in pipeline(train['en_f'], do_sample=True, ):
    results.append(out[0]['generated_text'])

display(timeSince(start, 1))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'2m 37s (- 0m 0s)'

In [204]:
for n in range(len(results)):
    res = results[n].split('(French sentence): ')
    print("> ", pairs.iloc[n,0])
    print("= ", pairs.iloc[n,1])
    print("< ", normalizeString(res[-1]))
    print("")



>  response to date there have been seventeen referrals considered by the committee
=  reponse il y a eu dix sept renvois qui ont ete examines par le comite jusqu a maintenant
<  au date il y a seventeen referrals qui ont ete examinees par le comite

>  verify and process bird banding data received from banders and update the bird banding database
=  verification et traitement des donnees sur le baguage des oiseaux recues des bagueurs et mise a jour de la base de donnees sur le baguage des oiseaux
<  verifiez et procedez a l analyse des donnees de bandage des oiseaux recus des banders et mis a jour la base de donnees de bandage des oiseaux

>  loneliness sadness boredom sleep problems and pain are just some of the age related physical psychological social and environmental stresses seniors can face
=  l isolement la tristesse l ennui l insomnie et la douleur sont autant de facteurs susceptibles d alimenter la detresse physique morale ou sociale des aines
<  la solitude la tristesse la 

## trying running from api

In [205]:
!pip install replicate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 12.1 MB/s eta 0:00:00


In [222]:
# go to the replicate link in discord resources and paste your api key here
REPLICATE_API_TOKEN=''

In [234]:
df.sample(1).iloc[0,0]

'bacteria grown at reduced temperature had a mortality of'

In [263]:
import replicate
replicate = replicate.Client(api_token=REPLICATE_API_TOKEN)

start = time.time()
for p in range(len(pairs)):
    pair = pairs.iloc[p]


    output = replicate.run(
        "meta/llama-2-7b-chat:13c3cdee13ee059ab779f0291d29054dab00a47dad8261375654de5540165fb0",
        input={"prompt": pair['en'],
            "system_prompt": "Format the response like this: (English sentence):(French sentence)"}

    )
    # The meta/llama-2-7b-chat model can stream output as it's running.
    # The predict method returns an iterator, and you can iterate over that output.
    print("> ", pair['en'])
    print('= ', pair['fr'])
    res = []
    for item in output:
        # https://replicate.com/meta/llama-2-7b-chat/versions/13c3cdee13ee059ab779f0291d29054dab00a47dad8261375654de5540165fb0/api#output-schema
        res.append(item)
    print(' '.join(res).split("sentence )")[-1].strip())
    print("")

display(timeSince(start, 1))

>  response to date there have been seventeen referrals considered by the committee
=  reponse il y a eu dix sept renvois qui ont ete examines par le comite jusqu a maintenant
:  Les  XVII  ré fér é ls  ont  été  exam in és  par  le  com ité .

>  verify and process bird banding data received from banders and update the bird banding database
=  verification et traitement des donnees sur le baguage des oiseaux recues des bagueurs et mise a jour de la base de donnees sur le baguage des oiseaux
V ér if iez  et  proc é de z  à  la  v ér ification  des  données  de  marc ott age  d ' o ise aux  re ç us  des  band ers  et  mise  à  jour  la  base  de  données  de  marc ott age  des  o ise aux .

>  loneliness sadness boredom sleep problems and pain are just some of the age related physical psychological social and environmental stresses seniors can face
=  l isolement la tristesse l ennui l insomnie et la douleur sont autant de facteurs susceptibles d alimenter la detresse physique morale ou

'0m 13s (- 0m 0s)'